In [0]:
# encoding=utf-8
# https://github.com/zhanlaoban/EDA_NLP_for_Chinese
import jieba
import synonyms
import random
from random import shuffle

ModuleNotFoundError: ignored

In [0]:
import pandas as pd
from tqdm import tqdm

In [0]:
import os
path = "/content/drive/My Drive/NLP/sentiment_compete"
os.chdir(path)

In [0]:
!pip install -U synonyms

     |████████████████████████████████| 54.5MB 73kB/s 
  Created wheel for synonyms: filename=synonyms-3.10.2-cp36-none-any.whl size=54555990 sha256=52e90935f3ddefd567f1ab2bed60bb73d8cddda007c921811a81ca959497bba4
  Stored in directory: /root/.cache/pip/wheels/35/34/8c/195ac92995b84be9e93dcc5ddbea940a1545bbca9a919e93b5
Successfully built synonyms


In [0]:
#停用词列表，默认使用哈工大停用词表
f = open('stop_words/hit_stopwords.txt')
stop_words = list()
for stop_word in f.readlines():
    stop_words.append(stop_word[:-1])

In [0]:
def synonym_replacement(words, replace_num):
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words])) # 非停用词
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms_words = synonyms.nearby(random_word)[0]
        if len(synonyms_words) >= 1:
            synonyms_word = random.choice(synonyms_words[1: min(5, len(synonyms_words))])
            new_words = [synonyms_word if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= replace_num:
            break

    new_sentence = ''.join(new_words)
    return new_sentence

In [0]:
def sentence_augument(sentence, num_aug = 1, alpha_sy = 0.2):
    seg_list = jieba.cut(sentence)
    seg_list = " ".join(seg_list)
    words = list(seg_list.split())
    num_words = len(words)
    replace_num = max(1, int(num_words *  alpha_sy))
    augmented_sentences = []
    for _ in range(num_aug):
        augmented_sentences.append(synonym_replacement(words, replace_num))
    return augmented_sentences

In [0]:
def data_augument(df, num_aug = 1):
    df['微博中文内容'] = df['微博中文内容'].astype(str)
    for index, line in tqdm(df.iterrows()):
        augument_sentences = sentence_augument(line['微博中文内容'], num_aug)
        for augment_line in augument_sentences:
            new_line = line.copy()
            new_line['微博中文内容'] = augment_line
            df = df.append(new_line, ignore_index=True)
    return df

In [0]:
text = '今天是个好日子,心想的事儿都能成,马上冲进top10'

In [0]:
data_augument(text)

TypeError: ignored

In [0]:
train_data = pd.read_csv('data/train_weibo.csv')

In [0]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81098 entries, 0 to 81097
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   微博id    81098 non-null  int64 
 1   微博发布时间  81098 non-null  object
 2   发布人账号   81098 non-null  object
 3   微博中文内容  80262 non-null  object
 4   微博图片    81098 non-null  object
 5   微博视频    81098 non-null  object
 6   情感倾向    81098 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 4.3+ MB


In [0]:
train_data['情感倾向'].value_counts()

 0    46163
 1    20397
-1    14538
Name: 情感倾向, dtype: int64

In [0]:
train_data_aug= data_augument(train_data, 1)

81098it [1:07:33, 20.00it/s]


In [0]:
train_data_aug['情感倾向'].value_counts()

 0    92326
 1    40794
-1    29076
Name: 情感倾向, dtype: int64

In [0]:
train_aug.to_csv('data/train_weibo_aug.csv', index=False)

In [0]:
train_data_neg = train_data[train_data['情感倾向'] == -1]
train_data_neg= data_augument(train_data_neg, 2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
14538it [13:45, 17.61it/s]


In [0]:
train_data_0 = train_data[train_data['情感倾向'] == 0]

In [0]:
train_aug = pd.concat([train_data_0, train_data_neg, train_data_1], ignore_index=True)

In [0]:
from sklearn.utils import shuffle
train_aug = shuffle(train_aug)

In [0]:
train_aug['情感倾向'].value_counts()

 0    46163
-1    43614
 1    40794
Name: 情感倾向, dtype: int64

In [0]:
train_aug.to_csv('data/train_weibo_aug.csv', index=False)

In [0]:
train_df = pd.read_csv('data/train_labled_yuanshi.csv')
test_df = pd.read_csv('data/10k_test.csv')

In [0]:
import re
def clean(text):
    text = re.sub(r"(回复)?(//)?\s*@\S*?\s*(:| |$)", " ", text)  # 去除正文中的@和回复/转发中的用户名
    text = re.sub(r'//@.*?:',"",text)
    text = re.sub(r"\[\S+\]", "", text)      # 去除表情符号
    # text = re.sub(r"#\S+#", "", text)      # 保留话题内容
    URL_REGEX = re.compile(
        r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))',
        re.IGNORECASE)
    text = re.sub(URL_REGEX, "", text)       # 去除网址
    text = text.replace("转发微博", "")       # 去除无意义的词语
    text = re.sub(r"\s+", " ", text) # 合并正文中过多的空格
    return text.strip()

In [0]:
test_df['微博中文内容'] = test_df['微博中文内容'].astype(str).apply(clean)

In [0]:
train_df['微博中文内容'] = train_df['微博中文内容'].astype(str).apply(clean)

In [0]:
train_df = train_df[train_df['情感倾向'].isin(['0', '-1', '1'])]

In [0]:
train_df = train_df.to_csv('data/train_clean.csv', index=False)
test_df = test_df.to_csv('data/test_clean.csv', index=False)

In [0]:
for index, row in tqdm(test_df.iterrows()):
  temp = train_df[train_df['微博中文内容'] == row['微博中文内容']]
  if temp.shape[0] > 0:
    test_df.loc[index, '情感倾向'] = temp.iloc[0]['情感倾向']

10000it [02:27, 67.72it/s]


In [0]:
need_train = test_df[test_df['情感倾向'] == -2]

In [0]:
test_labled = test_df[test_df['情感倾向'] != -2]

In [0]:
need_train.shape

(8256, 7)

In [0]:
test_labled.shape

(1744, 7)

In [0]:
test_labled['情感倾向'].value_counts()

0     1111
1      463
-1     170
Name: 情感倾向, dtype: int64

In [0]:
need_train.to_csv('data/need_test.csv', index=False)
test_labled.to_csv('data/test_labled.csv', index=False)

In [0]:
need_train['微博中文内容'].astype(str)

0       #你好2020#新年第一天元气满满的早起出门买早饭结果高估了自己抗冻能力回家成功冻发烧（大概...
1       大宝又感冒鼻塞咳嗽了，还有发烧。队友加班几天不回。感觉自己的情绪在家已然是随时引爆的状态。情...
2                           还要去输两天液，这天也太容易感冒发烧了，一定要多喝热水啊?
3                                 我太难了别人怎么发烧都没事就我一检查甲型流感?
4       果然是要病一场的喽回来第三天开始感冒今儿还发烧了喉咙眼睛都难受的一匹怎么样能不经意让我的毕设...
                              ...                        
9994    【#自然科学基金启动新型肺炎研究#：拟资助20项，每项150万】1月22日，国家自然科学基金...
9996    心灵鸡汤#武汉加油#我们所有人，和我们这个国家一起，正在经历着一场这个星球上史无前例的考验...
9997    武大人民医院：发热咳嗽并非新冠肺炎的唯一首发症状(来自@界面新闻)武汉大学人民医院研究组1月...
9998                                闭关第二天发现一根白发2019-nCoV?
9999             //@高反台:昨天还在想如果有动画短片就好了，今天就有了，视频果然更直接有效吧。
Name: 微博中文内容, Length: 8256, dtype: object

In [0]:
need_train['微博中文内容'] = need_train['微博中文内容'].astype(str).apply(clean)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
need_train['微博中文内容']

0       #你好2020#新年第一天元气满满的早起出门买早饭结果高估了自己抗冻能力回家成功冻发烧（大概...
1       大宝又感冒鼻塞咳嗽了，还有发烧。队友加班几天不回。感觉自己的情绪在家已然是随时引爆的状态。情...
2                           还要去输两天液，这天也太容易感冒发烧了，一定要多喝热水啊?
3                                 我太难了别人怎么发烧都没事就我一检查甲型流感?
4       果然是要病一场的喽回来第三天开始感冒今儿还发烧了喉咙眼睛都难受的一匹怎么样能不经意让我的毕设...
                              ...                        
9994    【#自然科学基金启动新型肺炎研究#：拟资助20项，每项150万】1月22日，国家自然科学基金...
9996    心灵鸡汤#武汉加油#我们所有人，和我们这个国家一起，正在经历着一场这个星球上史无前例的考验...
9997                          武大人民医院：发热咳嗽并非新冠肺炎的唯一首发症状(来自
9998                                闭关第二天发现一根白发2019-nCoV?
9999                    昨天还在想如果有动画短片就好了，今天就有了，视频果然更直接有效吧。
Name: 微博中文内容, Length: 8256, dtype: object

In [0]:
train_df = train_df[train_df['情感倾向'].isin(['0', '-1', '1'])]

In [0]:
train_df['微博中文内容'] = train_df['微博中文内容'].astype(str).apply(clean)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
train_df.to_csv('data/train_clean.csv', index=False)